# Chapter 27 of HS, Mineral Fuel, Oil, etc; Bitumin Subst; Mineral Wax Analysis

### Libraries

In [12]:
import requests
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from io import StringIO
from dataclasses import dataclass
from pathlib import Path
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import zipfile
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pmdarima import auto_arima
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_percentage_error
from scipy.stats import norm

In [2]:
import warnings

# Set the default behavior for warnings to ignore
warnings.filterwarnings("ignore")


### Datahandler class 
- deals with data, writing to and reading from CSV Files

In [3]:
class DataHandler:
    @staticmethod
    def to_parquet(dataframe, parquet_filename):
        # Create 'data' directory if it doesn't exist
        data_dir = os.path.join(os.getcwd(), 'data')
        os.makedirs(data_dir, exist_ok=True)

        # Writing dataframe to Parquet file within 'data' directory
        parquet_path = os.path.join(data_dir, parquet_filename)
        dataframe.to_parquet(parquet_path, index=False)

    @staticmethod
    def from_parquet(parquet_filename):
        # Reading Parquet file into a DataFrame
        parquet_path = os.path.join(os.getcwd(), 'data', parquet_filename)
        dataframe = pd.read_parquet(parquet_path)
        return dataframe


### Data Class 
- for Universal Objects

In [4]:
@dataclass
class data:
    I: object 
    E: object 

### TradeDataFilter class
- Filter to Extract Subsets

In [5]:
class TradeDataFilter:
    def __init__(self, data_frame):
        self.data_frame = data_frame

    @staticmethod
    def extract_state(dist_name):
        parts = dist_name.split(',')
        if len(parts) > 1:
            return parts[-1].strip()
        else:
            return dist_name

    def filter_data(self, **kwargs):
        filtered_data = self.data_frame.copy()

        for key, value in kwargs.items():
            if key == 'start_date':
                filtered_data = filtered_data[filtered_data['Date'] >= value]
            elif key == 'end_date':
                filtered_data = filtered_data[filtered_data['Date'] <= value]
            elif key in filtered_data.columns:
                if isinstance(value, list):
                    # Handle list values (e.g., for multiple commodities, countries, etc.)
                    filtered_data = filtered_data[filtered_data[key].isin(value)]
                else:
                    # Handle single value
                    filtered_data = filtered_data[filtered_data[key] == value]
            else:
                print(f"Warning: '{key}' column not found in data. Skipping filtering by this condition.")

        if filtered_data.empty:
            print("Error: Filtered data is empty.")
            return None
        else:
            return filtered_data.reset_index(drop=True)


### Fetching Data 

In [6]:

dh = DataHandler()
d = data(
            I = dh.from_parquet('ImportsData.parquet'),
            E = dh.from_parquet('ExportsData.parquet')
         )


ectypes = {
    'E_COMMODITY_SDESC' : str,
    'COMM_LVL' : str,
    'DISTRICT': int,
    'DIST_NAME': str,
    'CTY_CODE': int,
    'CTY_NAME': str,
    'ALL_VAL_MO':np.int64,
    'YEAR': int,
    'MONTH':int,
    'E_COMMODITY':np.int64,
    'SUMMARY_LVL': str,
    'SUMMARY_LVL2': str
}

ictypes = {
    'I_COMMODITY_SDESC' : str,
    'COMM_LVL' : str,
    'DISTRICT': int,
    'DIST_NAME': str,
    'CTY_CODE': int,
    'CTY_NAME': str,
    'GEN_VAL_MO':np.int64,
    'CON_VAL_MO':np.int64,
    'YEAR': int,
    'MONTH':int,
    'I_COMMODITY':np.int64,
    'SUMMARY_LVL': str,
    'SUMMARY_LVL2': str
}

# Convert data types accordingly
d.E = d.E.astype(ectypes)

# Convert data types accordingly
d.I = d.I.astype(ictypes)

d.I['US_STATE'] = d.I.DIST_NAME.apply(TradeDataFilter.extract_state)
d.E['US_STATE'] = d.E.DIST_NAME.apply(TradeDataFilter.extract_state)


d.I['CTYi'] = d.I.CTY_NAME.apply(lambda x: x + '(i)')
d.E['CTYe'] = d.E.CTY_NAME.apply(lambda x: x + '(e)')

d.I['Date'] = pd.to_datetime(d.I.YEAR.astype(str) + '-' + d.I.MONTH.astype(str), format='%Y-%m')
d.E['Date'] = pd.to_datetime(d.E.YEAR.astype(str) + '-' + d.E.MONTH.astype(str), format='%Y-%m')


fi = TradeDataFilter(d.I)
fe = TradeDataFilter(d.E)

In [7]:


class ForecastPlot:
    def __init__(self, data, split):
        self.data = data
        self.split = split

    def plot_forecast(self, original_data, forecast_values, lower_bound, upper_bound,
                      title='Time Series Forecasting', xlabel='Date'):
        fig = go.Figure()

        # Plot original data
        fig.add_trace(go.Scatter(
            x=original_data.index,
            y=original_data,
            mode='lines',
            name='Original Data'
        ))

        # Plot forecast
        fig.add_trace(go.Scatter(
            x=pd.date_range(start=original_data.index[-1], periods=len(forecast_values)+1, freq='MS')[1:],
            y=forecast_values,
            mode='lines',
            name='Forecast'
        ))

        # Plot prediction intervals
        fig.add_trace(go.Scatter(
            x=pd.date_range(start=original_data.index[-1], periods=len(lower_bound)+1, freq='MS')[1:],
            y=lower_bound,
            mode='lines',
            name='Lower Bound'
        ))

        fig.add_trace(go.Scatter(
            x=pd.date_range(start=original_data.index[-1], periods=len(upper_bound)+1, freq='MS')[1:],
            y=upper_bound,
            mode='lines',
            name='Upper Bound'
        ))

        fig.update_layout(
            title=title,
            xaxis_title=xlabel,
            yaxis_title='Values',
            font=dict(size=14, color='#000000'),
            legend=dict(
                orientation='h', 
                yanchor='bottom',
                y=1.02,
                xanchor='right',
                x=1,
                # bgcolor='rgba(0, 0, 0, 0.7)',  
                bordercolor='rgba(255, 255, 255, 0.5)',  
                borderwidth=1,                        
                font=dict(size=12, color='#000000'),   
                
            ),
                height= 600,
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            xaxis=dict(
                gridcolor='#666666', 
                zerolinecolor='#666666'
            ),
            yaxis=dict(
                gridcolor='#666666', 
                zerolinecolor='#666666'  
            )
        )

        fig.show()


class SARIMAModel(ForecastPlot):
    
    def __init__(self, data, column, split = 0.8):
        super().__init__(data, split)
        self.rvColumn = column
        self.model = None
        self.results = None
        self.mae_error = None
        self.mse_error = None
        self.r2 = None
        self.rmse = None
        self.mape = None

    def train_model(self, order, seasonal_order):
        self.model = sm.tsa.SARIMAX(self.data[self.rvColumn], order=order, seasonal_order=seasonal_order)
        self.results = self.model.fit(disp=0)


    def test_metrics(self, order, seasonal_order):
        self.tr, self.ts = train_test_split(self.data, train_size=self.split, shuffle=False)
        mdl = sm.tsa.SARIMAX(self.tr[self.rvColumn], order=order, seasonal_order=seasonal_order)
        rs = mdl.fit(disp=0)

        fc = rs.get_forecast(steps=len(self.ts))
        fcv = fc.predicted_mean

        self.mae_error = mean_absolute_error(self.ts, fcv)
        self.mse_error = mean_squared_error(self.ts, fcv)
        self.r2 = r2_score(self.ts, fcv)
        self.rmse = root_mean_squared_error(self.ts, fcv)
        self.mape = mean_absolute_percentage_error(self.ts, fcv)

    def plot_forecast(self, n_months, title='Time Series Forecasting', xlabel='Date'):
        forecast = self.results.get_forecast(steps=n_months)
        forecast_values = forecast.predicted_mean
        confidence_intervals = forecast.conf_int()
        
        super().plot_forecast(self.data[self.rvColumn], forecast_values, 
                              confidence_intervals.iloc[:, 0], confidence_intervals.iloc[:, 1],
                              title=title, xlabel=xlabel)


class ARIMAModel(ForecastPlot):
    
    def __init__(self, data, column, split=0.8):
        super().__init__(data, split)
        self.rvColumn = column
        self.model = None
        self.results = None
        self.mae_error = None
        self.mse_error = None
        self.r2 = None
        self.rmse = None
        self.mape = None

    def train_model(self, order):
        self.model = sm.tsa.ARIMA(self.data[self.rvColumn], order=order)
        self.results = self.model.fit()

    def test_metrics(self, order):
        self.tr, self.ts = train_test_split(self.data, train_size=self.split, shuffle=False)
        mdl = sm.tsa.ARIMA(self.tr[self.rvColumn], order=order)
        rs = mdl.fit()

        fc = rs.get_forecast(steps=len(self.ts))
        fcv = fc.predicted_mean
        self.mae_error = mean_absolute_error(self.ts, fcv)
        self.mse_error = mean_squared_error(self.ts, fcv)
        self.r2 = r2_score(self.ts, fcv)
        self.rmse = root_mean_squared_error(self.ts, fcv)
        self.mape = mean_absolute_percentage_error(self.ts, fcv)

    def plot_forecast(self, n_months, title='Time Series Forecasting', xlabel='Date'):
        forecast = self.results.get_forecast(steps=n_months)
        forecast_values = forecast.predicted_mean
        confidence_intervals = forecast.conf_int()
        
        super().plot_forecast(self.data[self.rvColumn], forecast_values, 
                              confidence_intervals.iloc[:, 0], confidence_intervals.iloc[:, 1],
                              title=title, xlabel=xlabel)
        


class TimeSeriesAnalyzer:
    def __init__(self, data, seasonal=True, m=12, trace=True, suppress_warnings=True):
        self.data = data
        self.seasonal = seasonal
        self.m = m
        self.trace = trace
        self.suppress_warnings = suppress_warnings
        self.best_arima_order = None
        self.best_sarima_order = None

    def find_best_orders(self):
        stepwise_fit = auto_arima(self.data, 
                                  seasonal=self.seasonal,
                                  m=self.m,
                                  trace=self.trace,
                                  suppress_warnings=self.suppress_warnings)

        # Get the best ARIMA and SARIMA orders
        self.best_arima_order = stepwise_fit.get_params()['order']
        self.best_sarima_order = stepwise_fit.get_params()['seasonal_order']

    def display_best_orders(self):
        print("Best ARIMA Order:", self.best_arima_order)
        print("Best SARIMA Order:", self.best_sarima_order)



import plotly.graph_objects as go

def create_circular_percentage_chart(percentage, title='Circular Percentage Chart'):
    # Create the figure for the circular gauge
    fig = go.Figure(go.Indicator(
        mode="gauge+number",  # Set the mode to display gauge and number
        value=percentage,  # Set the value (percentage) to be displayed
        title={'text': title, 'font': {'size': 20}},  # Set the title of the chart with larger font size
        gauge={'axis': {'range': [None, 100], 'tickwidth': 2, 'tickcolor': "gray", 'ticklen': 10, 'tickfont': {'size': 12}},  # Customize the axis ticks
               'bar': {'color': "steelblue"},  # Set the color of the gauge bar
               'bgcolor': "white",  # Set the background color of the gauge
               'borderwidth': 2,  # Set the border width
               'bordercolor': "lightgray",  # Set the border color
               'steps': [
                   {'range': [0, 100], 'color': "lightblue"}],  # Set the color range for the gauge
               'threshold': {'line': {'color': "red", 'width': 4},  # Set the threshold line color and width
                             'thickness': 0.75,  # Set the thickness of the threshold line
                             'value': percentage}}  # Set the threshold value (same as percentage)
    ))

    # Update layout properties for the figure
    fig.update_layout(
        height=300,  # Set the height of the chart
        width=300,  # Set the width of the chart
        margin=dict(t=50, b=50, l=50, r=50),  # Set margins for better spacing
        font={'color': 'black', 'family': 'Arial'},  # Set font color and family
        plot_bgcolor='rgba(0,0,0,0)',  # Set plot background color
        paper_bgcolor='rgba(0,0,0,0)',  # Set paper background color
    )

    # Show the chart
    fig.show()

# Example usage: Display a circular percentage chart with 75% filled
create_circular_percentage_chart(75, title='My Circular Percentage Chart')



In [8]:
rvColImports = 'GEN_VAL_MO'
rvColExports = 'ALL_VAL_MO'

# Example usage:
filteredDataI = fi.filter_data(I_COMMODITY=27)
filteredDataI = pd.DataFrame(filteredDataI.groupby(['Date']).sum()[rvColImports])

# Example usage:
filteredDataE = fe.filter_data(E_COMMODITY=27)
filteredDataE = pd.DataFrame(filteredDataE.groupby(['Date']).sum()[rvColExports])

In [9]:


tsANA_E = TimeSeriesAnalyzer(filteredDataE[rvColExports])
tsANA_E.find_best_orders()

tsANA_I = TimeSeriesAnalyzer(filteredDataI[rvColImports])
tsANA_I.find_best_orders()


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=5923.319, Time=0.85 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=5916.237, Time=0.02 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=5919.913, Time=0.07 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=5919.813, Time=0.08 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=5915.362, Time=0.01 sec
 ARIMA(0,1,0)(1,0,0)[12] intercept   : AIC=5917.936, Time=0.04 sec
 ARIMA(0,1,0)(0,0,1)[12] intercept   : AIC=5917.802, Time=0.04 sec
 ARIMA(0,1,0)(1,0,1)[12] intercept   : AIC=5918.200, Time=0.10 sec
 ARIMA(1,1,0)(0,0,0)[12] intercept   : AIC=5918.240, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[12] intercept   : AIC=5918.280, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[12] intercept   : AIC=5920.161, Time=0.03 sec

Best model:  ARIMA(0,1,0)(0,0,0)[12]          
Total fit time: 1.309 seconds
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=inf, Time=0.95 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   

#### ARIMA/SARIMA on Exports

In [10]:
srm = SARIMAModel(data=filteredDataE, column=rvColExports)
srm.train_model(order=tsANA_E.best_arima_order, seasonal_order=tsANA_E.best_sarima_order)
srm.plot_forecast(n_months=12)
srm.test_metrics(order=tsANA_E.best_arima_order, seasonal_order=tsANA_E.best_sarima_order)



arm = ARIMAModel(data=filteredDataE, column=rvColExports)
arm.train_model(order=tsANA_E.best_arima_order)
arm.plot_forecast(n_months=12)
arm.test_metrics(order=tsANA_E.best_arima_order)


#### ARIMA/SARIMA on Imports

In [11]:



srm = SARIMAModel(data=filteredDataI, column=rvColImports)
srm.train_model(order=tsANA_I.best_arima_order, seasonal_order=tsANA_I.best_sarima_order)
srm.plot_forecast(n_months=12)
srm.test_metrics(order=tsANA_I.best_arima_order, seasonal_order=tsANA_I.best_sarima_order)




arm = ARIMAModel(data=filteredDataI, column=rvColImports)
arm.train_model(order=tsANA_I.best_arima_order)
arm.plot_forecast(n_months=12)
arm.test_metrics(order=tsANA_I.best_arima_order)


In [13]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from xgboost import XGBRegressor

class XGBoostForecast(ForecastPlot):
    def __init__(self, data, targetColumn, lags=5, split_percent=0.8):
        super().__init__(data, split=split_percent)
        self.lags = lags
        self.targetColumn = targetColumn
        self.model = None
        self.mae_error = None
        self.mse_error = None
        self.r2 = None
        self.rmse = None
        self.mape = None
        self.std_error = None
        self.lBound = []
        self.uBound = []

    def series_to_supervised(self, dropnan=True): 
        for i in range(self.lags, 0, -1):
            self.data['lag_'+str(i)] = self.data[self.targetColumn].shift(i)

        if dropnan:
            self.data.dropna(inplace=True)


    def train(self):
        X_train, y_train = self.data.drop(self.targetColumn, axis=1), self.data[self.targetColumn]
        self.model = XGBRegressor()
        self.model.fit(X_train, y_train)

    def test_metrics(self):
        spltData = self.data.copy()
        split_index = int(len(spltData) * self.split)
        train, test = spltData.iloc[:split_index], spltData.iloc[split_index:]

        # Separate features and target variable
        X_train, y_train = train.drop(self.targetColumn, axis=1), train[self.targetColumn]
        X_test, y_test = test.drop(self.targetColumn, axis=1), test[self.targetColumn]

        # Initialize XGBoost regressor, fit and predict
        xgb_model = XGBRegressor()
        xgb_model.fit(X_train, y_train)
        y_pred = xgb_model.predict(X_test)

        self.mae_error = mean_absolute_error(y_test, y_pred)
        self.mse_error = mean_squared_error(y_test, y_pred)
        self.r2 = r2_score(y_test, y_pred)
        self.rmse = mean_squared_error(y_test, y_pred, squared=False)
        self.mape = mean_absolute_percentage_error(y_test, y_pred)
        # Calculate standard error
        residuals = y_test - y_pred
        self.std_error = np.std(residuals)





    def multi_step_forecast(self, confidence = 0.95, f_horizon=12):
        z = np.round(norm.ppf((1 + confidence) / 2), 2)
        print(z)
        for fh in range(f_horizon):
            next_month = self.data.index[-1] + pd.DateOffset(months=1)
            new_row = pd.Series(index=self.data.columns)
            self.data.loc[next_month] = new_row

            for lag in range(self.lags, 1, -1):
                self.data.loc[self.data.index[-1], 'lag_'+str(lag)] = self.data.iloc[-2]['lag_'+str(lag-1)]
            self.data.loc[self.data.index[-1], 'lag_1'] = self.data.iloc[-2][self.targetColumn]

            self.data.loc[self.data.index[-1], self.targetColumn] = self.model.predict(self.data.iloc[[-1]].drop(self.targetColumn, axis=1))[0]
            self.lBound.append(self.data.loc[self.data.index[-1], self.targetColumn] - z * self.std_error * np.sqrt(fh))
            self.uBound.append(self.data.loc[self.data.index[-1], self.targetColumn] + z * self.std_error * np.sqrt(fh))

    def plot_forecast(self, f_horizon = 12, title='XGBoost Forecasting', xlabel='Date'):

        forecast_values = self.data[self.targetColumn][-f_horizon:]
        
        super().plot_forecast(self.data[self.targetColumn][:-f_horizon],
                              forecast_values=forecast_values,
                              title='XGBoost Forecast',
                              xlabel=xlabel,
                              lower_bound=self.lBound,
                              upper_bound=self.uBound
                              )




fHorizon = 18
xgI = XGBoostForecast(data=filteredDataI.copy(), targetColumn='GEN_VAL_MO', lags=5, split_percent=0.8)
xgI.series_to_supervised(dropnan=True)
xgI.test_metrics()
xgI.train()
xgI.multi_step_forecast(confidence=0.95, f_horizon=fHorizon)
xgI.plot_forecast(f_horizon=fHorizon)



1.96


3764440483.1252365